# NMF Rank-2 con dataset Reuters

Autor: Benjamin Ayancán, PUC Chile
Tutor: Denis Parra, PUC Chile

En el siguiente notebook desarrollaremos un ejemplo de cómo poder generar una descomposición jerarquica a través del algoritmo de rango bajo NMF rank-2.

* Para este tutorial usaremos el dataset de noticias Reuters que nos entrega la libería `keras`

* Los aspectos teoricos y detalles del algoritmo son abordados en el  siguiente [notebook de Observablehq](https://observablehq.com/@beayancan/descomposicion-jerarquica)

* Para hacer más pedagógico el ejemplo se va a reducir tanto la cantidad de datos (filas) como la cantidad de features (columnas)

---

## Configuración

### Notebooks

--> Si lo estás ejecutando en un `jupyter notebook` debes instalar las siguientes librerías

```py
!pip install tensorflow
!pip install keras
```

--> Si lo estás ejecutando en `colab`



### Librerías

In [137]:
# Chequeamos que nuestro soporte no tenga problemas

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [138]:
# importamos las liberías que vamos a utilizar
# herramientas y el dataset
import os
import sys
import json
import keras
import statistics
import collections
import numpy as np
from random import randrange
from keras.datasets import reuters

---

## Reuters

#### ¿Por qué usar esta librería?

Reuters es un repositorio de documentos que podemos utilizar a través de `keras` o bien descargarlo en su [página web](http://konect.cc/networks/gottron-reuters/). Esta posee las siguientes características

* Son un conjunto de 11.228 noticias, etiquetados en 46 tópicos.


* Está preprocesado según un ranking de palabras de un volabulario, es decir su representación es a través de las posiciones en el ranking de las palabras de un documento
* Posee funciones sencillas que nos ayudan a manejar cómo representar los documentos sin gastar mucho tiempo en su preprocesamiento
* Se enfoca en el contenido de un documento, las palabras que lo representan, en vez del mensaje que este entregue

---

#### Cargar el dataset

In [139]:
# PARAMETROS
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/reuters/load_data

# Cantidad de palabras significativas del vocabulario a usar

# num_words = None # incluirlas todas
num_words = 200 # solo tomaremos las 100 primeras palabras
skip_top = 75 # palabras que no consideraremos

# máximo N° de palabras para representar a un documento
max_len = None # todas
# max_len = 50

# Porcentaje de palabras para usar en el test set
test_split = 0.4 # train set: 97.5%, test set: 2.5%

In [140]:
# Cargamos los de datos de clasificación de noticias de Reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words,
                                                         maxlen=max_len,
                                                         test_split=test_split,
                                                         skip_top=skip_top)

In [141]:
# Revisamos cómo fueron cargados los datos

# y_train, y_test contendrán los labels de las clases

num_clases = max(y_train) + 1

print(f'Tamaño train set: {x_train.shape}')
print(f'Tamaño test set: {x_test.shape}')
print(f'Cantidad clases: {num_clases}')

Tamaño train set: (6736,)
Tamaño test set: (4492,)
Cantidad clases: 46


### Qué es lo que contienen
* Cada elemento del set contiene los índices de las palabras más utilizadas por un documento

* Las palabras están ordenadas según su frecuencia de aparición
* Además cada documento pertenece a un tópico en especifico
* Notar por el `x_train.shape` que las representaciones no tienen un largo especifico

In [142]:
# Mostremos algunos ejemplos
# for i in range(1,15, 3):
#   print(f'Doc: {i},  largo: {len(x_test[i])}, clase {y_test[i]}')
#   print(f'   contenido: {x_test[i]} \n')

In [143]:
mapping = ['cocoa','grain','veg-oil','earn','acq','wheat','copper','housing','money-supply',
           'coffee','sugar','trade','reserves','ship','cotton','carcass','crude','nat-gas',
           'cpi','money-fx','interest','gnp','meal-feed','alum','oilseed','gold','tin',
           'strategic-metal','livestock','retail','ipi','iron-steel','rubber','heat','jobs',
           'lei','bop','zinc','orange','pet-chem','dlr','gas','silver','wpi','hog','lead']

train_count = collections.Counter(y_train)
test_count = collections.Counter(y_test)
total_words = [statistics.mean([len(e) for e in x_train[y_train.flatten() == i]]) for i in range(46)]

print("         Las clases y sus estadísticas")
print("{:4s} {:20s} {:5s}  {:5s} {:7s}".format("Idx","Clase", "train", "test", "Palabras"))
for i in range(46):
   print("{:5d} {:20s} {:5d} {:5d}   {:6.2f}".format(i,mapping[i], train_count[i], test_count[i], total_words[i]))

         Las clases y sus estadísticas
Idx  Clase                train  test  Palabras
    0 cocoa                   41    26   227.10
    1 grain                  331   206   184.57
    2 veg-oil                 53    41   189.55
    3 earn                  2379  1593    89.53
    4 acq                   1441   982   137.61
    5 wheat                   11    11   215.82
    6 copper                  33    29   153.64
    7 housing                 14     5   196.00
    8 money-supply           108    69   190.17
    9 coffee                  81    45   220.44
   10 sugar                   94    60   199.59
   11 trade                  295   178   257.92
   12 reserves                37    25   170.32
   13 ship                   129    80   166.84
   14 cotton                  17    11   154.47
   15 carcass                 16    13   182.12
   16 crude                  317   226   217.26
   17 nat-gas                 29    22   148.07
   18 cpi                     54    32   150.19
 

### Qué es lo que representan
* Solo tenemos los índices, pero ¿de qué?
  * Son la posición rankeada de la palabra
* Usamos la indexación del vocabulario del dataset para identificar la palabra

In [144]:
# Tenemos el vocabulario con las palabras y sus indices
word_index = reuters.get_word_index(path="reuters_word_index.json")

# Generamos un diccionario con el contenido
index_to_word = { value+3 : key for key, value in word_index.items() }

# Indices reservados
index_to_word[0] = '-PAD-'   # 0: carpeta
index_to_word[1] = '-START-' # 1: inicio secuencia
index_to_word[2] = '-UNK-'   # 2: elemento no encontrado

len_index = len(index_to_word)

## Preprocesamiento

* Para poder realizar los calculos, necesitamos
  * Una estructura regular de datos
  * Que cada documento posea un mismo largo

* Pasaremos primero los datos a una menor dimensión reduciendo las palabras y eliminando lo innecesario

In [145]:
# Primero eliminaremos las referencias a elementos que no se encuentran
# dentro de las 1000 palabras más usadas en el vocabulario
# ademas de las entradas reservadas

  # Eliminamos
  # 0: '-PAD-'
  # 1: '-START-'
  # 2: '-UNK-'
  # 12: '3'
  # 17: 'reuter'

def filtrar_relevante(arreglo, por_eliminar=[0,1,2]):
  """
  Borra las palabras que pertenecen a los indices del array por_eliminar
  """
  return list(filter(lambda x: x not in por_eliminar, arreglo))

def eliminar_reservadas(x_array):
  """
  Eliminamos las entradas reservadas y entradas inutiles
  retorna el contenido homogeneo del doc
  """
  por_eliminar = [0,1,2,12,17]
  largo_test, = x_array.shape
  for i in range(largo_test):
    x_array[i] = filtrar_relevante(x_array[i], por_eliminar)
  return x_array

In [146]:
# Para hacer un ejemplo más sencillo de entender
# Selecionaremos solo las primeras 7 clases

def conteo_labels(n, p):
  return list( [0, randrange(p-3, p+3)] for _ in range(n) )

def reducir_labels(data_array, labels, k=9, pivote=25):
  """
  Filtramos los documentos que pertenezcan a las primeras k clases
  Retornamos el arreglo con los documentos y sus correspondientes labels
  """
  
  conteo = conteo_labels(max(labels), pivote)
  retorno, retorno_labels = list(), list()
  for i in range(len(data_array)):
    if labels[i] < k and conteo[labels[i]][0] < conteo[labels[i]][1]:
      retorno.append(data_array[i])
      retorno_labels.append(labels[i])
      conteo[labels[i]][0] += 1
  return np.array(retorno), retorno_labels

In [147]:
x_test = eliminar_reservadas(x_test)
x_data, y_data = reducir_labels(x_test, y_test, k=7, pivote=25)
print(x_data.shape)

(159,)


In [148]:
# Mostremos lo que contiene
for i in range(1,15, 3):
  #print(f'Doc: {i},  Largo: {len(x_data[i])}, Clase {y_data[i]}')
  #print(f' {x_data[i]}', '\n', ' '.join([index_to_word[word] for word in x_data[i]]), "\n")
  continue

### Pasar los datos a matriz doc-term

* Pasamos a una matriz $A \in \mathbb{R}^{m \times n}$ donde $m$ es la cantidad de documentos y $n$ es la cantidad de palabras del vocabulario
* Pasaremos los datos a una representación de $\{0, 1\}$
* Representando la entrada $A[i,j]$ la aparición en el $i$-ésimo documento la $j$-ésima palabra

In [149]:
# Importamos tokenizer para producirlo

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

tkn = Tokenizer(num_words=num_words) # tamaño del vocabulario
num_clases = max(y_data) + 1

In [150]:
# Contruimos la matriz pasando los datos a binario

x_data_bin = tkn.sequences_to_matrix(x_data, mode='binary')
y_data_cat = to_categorical(y_data, num_clases)

In [151]:
# Revisamos los como queda la representacion

entry = 10
print(f'Train Set: {x_data_bin.shape} \n {x_data_bin[entry]}\n')
print(f'Test Set: {y_data_cat.shape}   (usaremos este) \n {y_data_cat[entry]}')

Train Set: (159, 200) 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0.]

Test Set: (159, 7)   (usaremos este) 
 [0. 1. 0. 0. 0. 0. 0.]


In [152]:
# Siguiendo la interpretación del paper

A_matrix = np.transpose(x_data_bin)

# NMF Jerárquico

* Ya tenemos unos datos preprocesados de forma ideal para nuestro trabajo
* Realizaremos la secuencia de NMF jerárquico siguiendo el paper
  * [Fast Rank-2 Nonnegative Matrix Factorization for Hierarchical Document Clustering](https://smallk.github.io/papers/hierNMF2.pdf)

* Recordar que el objetivo es minimizar la siguiente operación
  $$\min_{W \geq 0, H \geq 0} ||A - WH||_2^{2}$$

* Dado que corresponde a un problema no convexo, dividimos la resolución en **dos poblemas convexos**
$$\min_{H \geq 0} ||A - WH||_2^{2}$$
$$\min_{W \geq 0} ||A^T - H^T W^T||_2^{2}$$

## NMF Rank-2

* Usaremos el algoritmo NNSL para generar una estructura de árbol binario jerárquico

* Una factorización de este tipo es de "mala calidad", pues no se obtiene una buena aproximación de la matriz factorizada. Sin embargo, la eficiencia del algoritmo consigue realizar la operación de forma rápida y sencilla.

* Utilizaremos el punto anterior para desarrollar la aproximación de **NMF Rank-2** de **forma recursiva** sobre la matriz para generar una solución adecuada y más interpretativa debido a su estructura.

### Algoritmo Genérico NNSL

Sea una matriz $A \in \mathbb{R}^{m \times n}$ compuesta por multiples columnas $[a_1, a_2, \ldots, a_n]$, con  $a_i \in \mathbb{R}^{m}$ independientes entre sí (supuesto), tenemos que es posible iterar de la siguiente manera para encontrar la matriz incognita derecha $X \in \mathbb{R}^{2 \times n}$ dado que conocemos la matriz izquierda $B \in \mathbb{R}^{m \times 2}$.

#### Input:
Recibimimos la matriz a factorizar $A$ junto a su ya conocida matriz izquierda de la factorización (supuesto)

* $A \in \mathbb{R}^{m \times n}, \qquad B = [b_1, b_2] \in \mathbb{R}^{m \times 2}$

#### Output:
Obtenemos la matriz derecha que minimiza la siguiente ecuación

* $X = \text{argmin}_{X \geq 0} ||BX - A||^2_2$

#### Pasos

* Resolvemos mínimos cuadrados $ \quad X = \text{argmin}_{X} ||BX - A|| \in \mathbb{R}^{2 \times n}$


* Es necesario asegurarnos de que la matriz será positiva

* Por lo que calculamos

    * $\quad u = \dfrac{<A^T, b_1>}{||b_1||^2} \in \mathbb{R}, \qquad v = \dfrac{<A^T, b_2>}{||b_2||^2} \in \mathbb{R}$


* for $j=1$ to $n$:
  * if $\quad X_j \geq 0 \quad \Longrightarrow \quad $ continue
  * else:
    * if $\quad u_1 ||b_1|| \geq v_j ||b_2|| \quad \Longrightarrow \quad X_j = [u_j, \, 0]^T$
    * else: $\quad \Longrightarrow \quad X_j = [0, \, v_j]^T$
    
    
* return $X$

### Código NNSL

A través del algoritmo anterior, es posible resolver la ecuación convexa

$$\text{min}_{X \geq 0} ||BX - Y||^2_2$$

Partimos realizando el código del algoritmo recien visto

In [153]:
def rank2(A, W):
  """
  Recibe las matrices objetivo A y su matriz izquierda W
  Calcula la resolución iterativa de la minimización según el paper
  y obtenemos la minimización de H a partir de W
  Retorna las matrices W, H de las descomposición
  """
  m, n = np.shape(A)

  # resolvemos por minimos cuadrados
  H = np.linalg.solve(np.dot(np.transpose(W), W), np.dot(np.transpose(W), A))

  # Separamos en columnas
  w1, w2 = W[:, 0], W[:, 1]
  beta1, beta2 = np.linalg.norm(w1), np.linalg.norm(w2)

  # normalizamos
  u, v = np.dot(np.transpose(A), w1)/beta1, np.dot(np.transpose(A), w2)/beta2

  for j in range(n):
    # Para cada vector determinamos si cumple con la solucion
    retorno_j = np.zeros(2)
    if (H[:, j] >= 0).all():
      continue
    elif u[j]*beta1 >= v[j]*beta2:
      retorno_j[0] = u[j]
    else:
      retorno_j[1] = v[j]
    H[:, j] = retorno_j
  return W, H

def NMF_rank2(A, W=None, H=None, k=2, **kwargs):
  """
  Recibe la matriz objetivo y matrices iniciales
  Se realiza dos veces la minimización primero para H
  y luego para W
  Retorna la descomposición W, H de baja calidad
  """
  m, n = np.shape(A)

  # Iniciamos las matrices
  if W is None:
    W = np.random.rand(m, k)

  if H is None:
    H = np.zeros((k, n))
  
  # Realizamos las minimizaciones
  W, H = rank2(A, W)
  HT, WT = rank2(np.transpose(A), np.transpose(H))
  # Retornamos los valores que resultaron minimizados
  return np.transpose(WT), np.transpose(HT)

Para hacer el calculo de NMF rank-2 vamos a utilizar diversas iteraciones en una operacion recursiva que busque ir ajustando la solución de acuerdo al calculo anterior, cada iteración para NMF se guiará de la siguiente forma

* Para resolver los problemas convexos planteados anteriormente resolveremos en torno a la matriz derecha $H$ del problema, ajustandola de acuerdo a la izquierda $W$ para aproximar el objetivo $A$ según el algoritmo NNSL

$$\min_{H \geq 0} ||A - WH||_2^{2}$$

* Tras esto, invertimos los papeles ajustando la matriz izquierda $W$ respecto de la matriz derecha $W$ usando el mismo algoritmo, esto lo conseguimos transponiendo las matrices ya que resuelven el mismo problema

$$\min_{W \geq 0} ||A^T - H^T W^T||_2^{2}$$


In [154]:
def calculo_NMF(A, max_iteraciones=15, k=2, W=None, H=None, error=0.5):
  """
  Recibe la matriz objetivo A, dimension k,
  máximo de iteraciones y matrices iniciales
  Realiza de forma recursiva la aplicación de rank-2
  para así obtener una mejor aproximación
  Retorna los elementos W, H que aproximan A
  al alcanzar una cota de error o superar el maximo
  """
  # Inicializamos las matrices
  m, n = np.shape(A)
  if W is None:
    W = np.random.rand(m, k)

  for i in range(max_iteraciones):
    W, H = rank2(A, W)
    HT, WT = rank2(np.transpose(A), np.transpose(H))
    W, H = np.transpose(WT), np.transpose(HT)
    if (np.linalg.norm(A - np.dot(W, H))) < error: break
  return W, H

* Para alcanzar la representación que busca el algoritmo, tenemos que la matriz W debe ser la frecuencia de las palabras, por lo que pasamos a normalizar sus columnas
* Del mismo modo ponderamos su fila respectiva en la matriz H para no perder representatividad

* Además de eso, debido a que el calculo es con inicializacion de matrices random, es necesario realizar el calculo de la aproximación en diversas veces, pues puede ocurrir que minimos cuadrados no posea solucion al no ser una matriz diagonal dominante, por lo que hay que empezar denuevo.

* Por medio de experimientos fue posible determinar que 15 es una cota adecuada de intentos a realizar con tal de conseguir finalmente la aproximacion que sea de buena mediada a la matriz objetivo

In [155]:
def normalizar_descomposicion(W, H):
  """
  Normaliza las columnas de W y pondera respectivamente
  las filas de H para el resultado esperado
  """

  for j in range(2):
    norma = np.linalg.norm(W[:, j])
    W[:, j] = W[:, j]/norma
    H[j, :] = H[j, :]*norma
  return W, H


def calcular_descomposicion(A_matrix, max_iteraciones=15, max_intentos=10):
  """
  Recibe matriz objetivo, cantidad maxima iteraciones e intentos de calcular
  Calcula la descomposición rank-2 de forma reiterativa
  Si el i-esimo intento alcanza la cota
  se retorna la descomposición W, H
  """
  salida, excepcion = False, False
  for i in range(max_intentos):
    try:
      W, H = calculo_NMF(A_matrix, max_iteraciones, k=2, W=None, H=None)
      error = np.linalg.norm(np.dot(W, H) - A_matrix)
      if error < 60:
        salida = True
    except:
      excepcion = True
    else:
      if not excepcion and salida:
        return normalizar_descomposicion(W, H)

## Generar Estructura jerárquica

Dado que ya tenemos lo suficiente para poder generar una factorización NMF de rango 2, vamos a definir **la forma de generar la estructura jerárquica de árbol binario**. Una idea general para esto es la siguiente

* Tomar la matriz $A$ como nodo raiz
* Realizar descomposición NMF rank-2 sobre ella obteniendo $W \in \mathbb{R}^{m \times 2}$ y $H \in \mathbb{R}^{2 \times n}$
    * Donde $W$ serán dos largas columnas con la distribución de palabras
    * Y $H$ serán dos largas filas con la distribución de documentos
* Determinamos a través de las columnas de $W$ si la descomposición NMF entregó una buena separación
    * Es necesario definir qué es una buena separación
* En caso de que que sea una buena separación, dividimos (*split*) la matriz raiz en dos ramas $A_{1}$ y $A_{2}$ a partir de elementos de $W$ y $H$. Repitiendo posteriormente el paso sobre cada una.

* O bien si no es una buena separación, olvidamos lo calculado y volvemos a empezar.


### Pensemos en formato árbol

Para el algoritmo es necesario destacar que cada nodo del árbol jerárquico estará representado por una matriz $A_{i} \in \mathbb{m_i \times n_i}$, el cómo es generado cada uno de estos nodos se mostrará en las siguientes celdas. Para hacer las divisiones usaremos la información de las palabras que se encuentra contenida en la matriz $W$ de cada nodo, aquí una guía de pasos.

Si yo aplico NMF sobre $A$ y genero las matrices $W$ y $H$, al seleccionar solo ciertos elementos de $W$ y de $H$, digamosles $W'$ y $H'$, multiplicandolos puedo generar una matriz nueva y más pequeña $W' \cdot H' = A'$. Y usando los elementos que no seleccioné antes y los agrupo como $W''$ y $H''$ también genero una matriz nueva más pequeña $W'' \cdot H'' = A''$.

La clave es que tanto $A'$ como $A''$ pueden llegar a ser diferentes (depende de cómo hayamos seleccionado) pero ambas contienen información de $A$.

* El dividir y conquistar los datos nos permitirá realizar el algoritmo de forma recursiva para poder realizar esta selección de buena manera
  * Aplicaremos NMF sobre A
  * Revisaremos si vale la pena dividir
  * Haremos split de los datos
  * A estos dos hijos de datos les aplicaremos NMF
  * Continuaremos hasta alcanzar cierto objetivo

## Pensemos cómo hacer el árbol

Denotaremos a $`\mathcal{N}$ como **un nodo cualquiera del árbol binario**, cuyo corazón es una matriz $A \in \mathbb{m \times n}$, de este nodo sabemos

* La matriz izquierda $W$ de NMF rank-2 contiene dos columnas con la distribucion de palabras

* Por lo anterior, $`\mathcal{N}$ tiene asociado una distribución de palabras de una columna de alguna matriz $W$, ya sea $W[1] o W[2]$, que permitió crear este nodo

* Denotaremos como $f_{\mathcal{N}}$ a la lista de palabras asociadas a $\mathcal{N}$ que se encuentran ordenadas según su frecuencia

* Si aplicamos NMF sobre $A$, generaremos dos **posibles hijos** para el nodo $\mathcal{N}$. De igual forma, estos hijos tendrán asociadas listas de palabras que llamaremos $f_{\mathcal{L}}$ y $f_{\mathcal{R}}$ por las columnas izquierda y derecha de la **posible** matriz $W$ de su NMF.

    * Es necesario destacar el **posible** por lo que se mencionó en pasos anteriores, el ejecutar NMF no te asegura que los hijos sean los adecuados
    
    
* A partir de estas listas de palabras generaremos una métrica que nos permitirá decidir si es conveniente o no dividir el nodo con estos hijos.

In [156]:
def idx_plbs(arreglo):
  """
  Recibe un arreglo de palabras
  Genera un diccionario con los detalles de la palabra
  retornando una lista ordenada según relevancia
  """
  largo = len(arreglo)
  retorno = list({'word': i, 'value': arreglo[i]} for i in range(largo))
  retorno = sorted(retorno, key=lambda x: x['value'], reverse=True)

  for i in range(largo):
    retorno[i]['id'] = i
  return retorno

def generar_arrays(array_N, array_L, array_R):
  """
  Recibe los arreglos para poder dividir
  Retorna los arreglos ordenados según relevancia de sus palabras
  """
  return idx_plbs(array_N), idx_plbs(array_L), idx_plbs(array_R)

* Ya tenemos los arreglos de las palabras ordenados
* Es necesario definir una métrica para saber si los arreglos nos permiten separar de forma exitosa
* En nuestro caso vamos a utilizar el **modified Normalized Discounted Cumulative Gain** o bien **mNDCG**

* Este nos permite decidir qué tan bien mantienen la frecuencia los arreglos $\mathcal{L}$ y $\mathcal{R}$ respecto de su nodo padre $\mathcal{N}$, es decir, si ganamos información nueva en caso de separar de esta forma

    * Piensa en cómo separas una canasta con muchas frutas en dos canastas, si quieres clasificarlas ¿Vas a colocar frutas de los mismos tipos en la misma proporción en las canastas?

In [157]:
def factor_descuento(word, array_L, array_R):
  """
  Recibe los arreglos y la palabra para la cual
  se va a calcular su descuento
  Retorna el descuento de la palabra
  """

  fi_L = next(x for x in array_L if x['word'] == word)
  fi_R = next(x for x in array_R if x['word'] == word)  
  return np.log2(len(array_L) - max(fi_L['id'], fi_R['id']) + 1)


def ganancia_palabra(word, array_N, array_L, array_R):
  """
  Recibe los arreglos y la palabra de la que se quiere obtener su ganancia
  Retorna la ganancia de la palabra
  """
  
  descuento = factor_descuento(word, array_L, array_R)
  elemento = next(x for x in array_N if x['word'] == word)
  return np.log2(len(array_L) - elemento['id'] + 1)/descuento


def ganancias(array_N, array_L, array_R):
  """
  Calcula la ganancia del arreglo
  Retorna el arreglo de las ganancias y ordenada según ganancia
  """
  retorno = list()
  for word in range(len(array_N)):
    gan_actual = ganancia_palabra(word, array_N, array_L, array_R)
    retorno.append({'palabra': word, 'ganancia': gan_actual})
  
  return retorno, sorted(retorno, key=lambda x: x['ganancia'], reverse=True)

In [158]:
def MDCG(gan_array):
  """
  Calculo de MDCG según el array que se entregue
  Retorna el valor de ganancia
  """
  largo = len(gan_array)
  elementos = list(gan_array[i]['ganancia']/np.log2(i+1) for i in range(1, largo))
  return gan_array[0]['ganancia'] + sum(elementos)


def mNDCG(gan_array, gan_sort):
  """
  Calculo del puntaje a través de los arrays listos
  """
  return MDCG(gan_array)/MDCG(gan_sort)


def puntaje(f_N, f_L, f_R):
  """
  Calcula el puntaje de la descomposición NMF actual
  Retorna el valor que nos ayuda a decidir
  """
  gan, gan_sort = ganancias(*generar_arrays(f_N, f_L, f_R))
  return mNDCG(gan, gan_sort)**2

In [159]:
def elem_puntaje(A_matrix, L_matrix, R_matrix):
  """
  Calcula la descomposición NMF de A (nodo)
  y de sus posibles hijos
  Retorna los elementos necesarios para determinar si conviene
  """

  condicion = False
  while not condicion:
    try:
      W, H = calcular_descomposicion(A_matrix)
      WL, HL = calcular_descomposicion(L_matrix)
      WR, HR = calcular_descomposicion(R_matrix)
      condicion = True
    except:
      condicion = False
    else:
      if condicion:
        return W, H, WL, HL, WR, HR

def calculo_puntajes(W, H, WL, HL, WR, HR, i):  
  """
  Calcula el puntaje de los hijos del nodo
  a partir de los 
  """
  X = W[:, i].copy()

  puntaje_N1 = puntaje(X, WL[:, 0], WL[:, 1])
  puntaje_N2 = puntaje(X, WR[:, 0], WR[:, 1])

  return puntaje_N1, puntaje_N2


def puntajes_hijos(A, L, R):
  """
  Genera el calculo del puntaje a partir de los
  elementos necesario a partir del nodo
  """
  # W, H, WL, HL, WR, HR = elem_puntaje(A, L, R)
  return calculo_puntajes(*elem_puntaje(A, L, R), 0)
  

In [160]:
def agregar_columna(A, columna):
  """
  Agrega columna a la matriz A sin importar su contenido
  Retorna la matriz con la columna añadida
  """
  if A is None:
    A = np.zeros((len(columna), 1))
    A[:, 0] = columna
  else:
    A = np.column_stack((A,columna))
  return A


def split_matrix(A_matrix, W, H, columnas):
  """
  Separación de la matriz por contenido
  Retorna la separación en dos matrices
  
  col_docs:
  """
  m, n = np.shape(A_matrix)

  A1, A2 = None, None
  
  retorno_A1 = list()
  retorno_A2 = list()

  for j in range(n):
    if H[0][j] > H [1][j]:
      A1 = agregar_columna(A1, A_matrix[:, j])
      retorno_A1.append(columnas[j])
    else:
      A2 = agregar_columna(A2, A_matrix[:, j])
      retorno_A2.append(columnas[j])

  if A1.shape[1] >= A2.shape[1]:
    return A1, A2, retorno_A1, retorno_A2
  return A2, A1, retorno_A2, retorno_A1


# Parte final

Vamos a generar un arreglo que contenga la estructura de nuestro arbol
* Será un ejemplo sencillo por lo que usaremos pocos nodos
* Usamos un arreglo para los nodos generado
* Retornaría este arreglo que describe la estructura

In [161]:
# Variables globales

numero_nodos = 7 # cantidad nodos para crear
beta = 1.1 # diferencia de tamaño mínima que habrá entre los nodos

In [162]:
def palabras_columna(W, i):
  """
  Genera el arreglo de las palabras de la columna i
  Retorna arreglo diccionarios con los datos ordenados
  """

  entradas = ['idx', 'value']
  distribucion, retorno = W[:, i], list()
  for item in enumerate(distribucion):
    retorno.append(dict(zip(entradas, item)))
  return sorted(retorno, key=lambda i: i['value'], reverse=True)


def encontrar_significado(arreglo):
  """
  Recibe el arreglo de indices de palabras
  Retorna los elementos con atributo word que es el significado
  """
  for i in range(len(arreglo)):
    arreglo[i]['word'] = index_to_word[arreglo[i]['idx'] + 4]
  return arreglo

def eliminar_numeros(arreglo):
  numeros = [str(i) for i in range(10)]
  retorno = list()
  for palabra in arreglo:
    if palabra['word'][0] in numeros:
      continue
    retorno.append(palabra)
  return retorno

def palabras_destacadas(W, cantidad=3):
  """
  Selecciona las palabras más relevantes
  de la matriz W
  Retorna un arreglo con astas palabras
  """
  n = int(np.round(cantidad/2))+1
  retorno = encontrar_significado(palabras_columna(W, 0)[:n])
  retorno = eliminar_numeros(retorno)
  retorno_2 = encontrar_significado(palabras_columna(W, 1)[:n])
  retorno_2 = eliminar_numeros(retorno_2)
  retorno.extend(retorno_2)
  return retorno

In [163]:
def obtener_palabras_destacadas(lista_arbol, cantidad=10):
  for nodo in lista_arbol:
    if nodo['matrix'].shape[1] == 1:
      nodo['W'] = nodo['matrix']
    elif 'W' not in nodo.keys():
      nodo['W'], nodo['H'] = calcular_descomposicion(nodo['matrix'])
    nodo['destacadas'] = palabras_destacadas(nodo['W'], cantidad)
    #nodo['destacadas'] = palabras_relevantes(nodo['destacadas'])
  return lista_arbol

In [164]:
def seleccionar_nodo(lista_nodos):
  """
  Recibe el arreglo de los nodos de la estructura
  Calcula cual nodo es conveniente separar y lo retorna
  """
  if len(lista_nodos) == 1:
    return lista_nodos[0]
  #lista_nodos = sorted(lista_nodos, key = lambda i: i['id'], reverse=False)
  lista_nodos = sorted(lista_nodos, key = lambda i: i['puntaje'], reverse=True)
  for elemento in lista_nodos:
    if elemento['hijos'] is None:
      return elemento

In [165]:
def jerarquizacion(A_matrix):
  """
  Genera la estructura de jerarquía realizando
  descomposiciones NMF de forma recursiva
  Retorna la estructura 
  """

  outliner = None
  lista_nodos = list()

  primer_nodo = {
    'id': 1,
    'parent': None,
    'matrix': A_matrix,
    'puntaje': 1,
    'shape': A_matrix.shape,
    'columnas': list(i for i in range(A_matrix.shape[1]))
  }

  lista_nodos.append(primer_nodo)

  for i in range(1, numero_nodos, 2):
    M = seleccionar_nodo(lista_nodos)
    M['W'], M['H'] = calcular_descomposicion(M['matrix'])
    #M['W'], M['H'] = W, H
    
    N1, N2, cols_N1, cols_N2 = split_matrix(M['matrix'],
                                            M['W'],
                                            M['H'],
                                            M['columnas'])
    
    puntaje_N1, puntaje_N2 = puntajes_hijos(M['matrix'], N1, N2)
    
    N1_nodo = {
    'id': i+1,
    'parent': M['id'],
    'matrix': N1,
    'puntaje': puntaje_N1,
    'shape': N1.shape,
    'hijos': None,
    'columnas': cols_N1
    }

    N2_nodo = {
    'id': i+2,
    'parent': M['id'],
    'matrix': N2,
    'puntaje': puntaje_N2,
    'shape': N2.shape,
    'hijos': None,
    'columnas': cols_N2
    }

    M['hijos'] = [i+1, i+2, ]

    lista_nodos.append(N1_nodo)
    lista_nodos.append(N2_nodo)

  return obtener_palabras_destacadas(lista_nodos, cantidad=10)
  #return lista_nodos

In [166]:
def matriz_W_lista(lista_nodos):
  for nodo in lista_nodos:
    if 'W' not in nodo.keys():
      nodo['W'], nodo['H'] = calcular_descomposicion(nodo['matrix'])
  return lista_nodos


def limpiar_lista(lista_nodos):
  elementos = ['matrix', 'W', 'H']
  new_list = [{k: v for k, v in d.items() if k not in elementos} for d in lista_nodos]
  return new_list

In [167]:
lista_arbol = jerarquizacion(A_matrix)

In [168]:
lista_arbol = obtener_palabras_destacadas(lista_arbol)

In [169]:
#lista_arbol

In [170]:
#lista_arbol = limpiar_lista(lista_arbol)

In [171]:
def topicos_relevantes(y_data, columnas, mapping):
  counts = dict()
  for col in columnas:
    if str(y_data[col]) in counts:
      counts[str(y_data[col])] += 1
    else:
      counts[str(y_data[col])] = 1
      
  keys = ['label', 'frequency']

  auxiliar = list(dict(zip(keys, tupla)) for tupla in counts.items())

  labels = sorted(auxiliar, key = lambda i: i['frequency'], reverse=True)
  
  for elemento in labels:
    elemento['label'] = int(elemento['label'])
    elemento['label_name'] = mapping[elemento['label']]
  return labels[:3]

def palabras_relevantes(palabras):
  return list(x['word'] for x in palabras[:5])

In [172]:
def presentar_nodos(lista_arbol):
  for objeto in lista_arbol:
    print(f"Nodo {objeto['id']}")
    print(f"  parent: {objeto['parent']} - leafs {objeto['hijos']}")
    topicos = topicos_relevantes(y_data, objeto['columnas'], mapping)
    for topico in topicos:
      print(topico)
    print("")
    palabras = palabras_relevantes(objeto['destacadas'])
    frase = ""
    for w in palabras:
      if frase:
        frase += " / "
      frase += w
    print(frase)
    print("----------------------------------------\n")

In [173]:
lista_arbol = obtener_palabras_destacadas(lista_arbol)

In [174]:
#lista_arbol

In [175]:
#presentar_nodos(lista_arbol)

In [176]:
import operator

def top_words(palabras):
  arreglo_palabras = dict((x['word'], x['value']) for x in palabras)
  return dict(sorted(arreglo_palabras.items(), key=operator.itemgetter(1), reverse=True))

def arreglar_destacadas(lista_nodos):
  for nodo in lista_nodos:
    nodo['destacadas'] = top_words(nodo['destacadas'])
  return lista_nodos

In [177]:
lista_arbol = arreglar_destacadas(lista_arbol)

In [178]:
def seleccionar_caracteristicas(lista_nodos, caracteristicas):
  retorno = list()
  for elemento in lista_nodos:
    actual = dict()
    for x in caracteristicas:
      actual[x] = elemento[x]
    retorno.append(actual)
  return retorno

In [179]:
lista_relevante = seleccionar_caracteristicas(lista_arbol, ['id', 'parent', 'hijos', 'destacadas'])

In [180]:
with open('./reuters.json', 'w') as file:
    json.dump(lista_relevante , file)